Title: Analyzing Temperature and Snowfall Change in the White Mountains
Author: Nick Querrazzi
3/28/2023
License: MIT -- https://opensource.org/licenses/MIT

Introduction:
    With climate change's impacts growing more prevalent each year, we must learn to use historical changes in our environment to prepare us for what the future may look like. This script analyzes any changes in both the temperature and the snowfall on Mt. Washington in the White Mountains of New Hampshire. The script will also look for any correlation between possible rising temperatures and a change in snowfall. The White Mountains are a popular winter destination for winter sport enthusiasts. If winters are becoming warmer and getting less snow, will it bring less tourists and therefore less money into the area? We must understand the trends in temperature and snowfall in order to protect our winters, if necessary. Not only does protecting our winters protect the economy and community in the White Mountains, but it also preserves the natural environment.

Hypothesis:
    It is hypothesized that there is an increasing trend in temperature and that it is leading to a decrease in snowfall in the White Mountains of New Hampshire throughout the winter (Dec 21-Mar 20).

Study Site:
    The data being analyzed was recorded on Mt. Washington in New Hampshire. The measurements took place at an elevation of 6272' and started in 1948 until 2023. The elevation is a few thousand feet higher in elevation than most the surrounding ski mountains, although this data provided the most thorough and longest dataset. While there may be differences in amount of snowfall and temperature, the trends this analysis provides will be applicable to the surrounding White Mountains. 

Data Set:
    The dataset obtained through NOAA (gauge: USW00014755) form Mt.Washington provides daily snowfall, minimum temperature, and maximum temperature each day from 1948 until 2023.
    Link to data: https://www.ncei.noaa.gov/access/search/data-search/normals-hourly?bbox=44.370,-71.403,44.170,-71.203

In [ ]:
#Load Data:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
from scipy import stats

#%% Specify Parameters

fig_title1 = 'Mt. Washington, NH Snow Fall'
fig_title2 = 'Mt. Washington, NH Average Temperature'

#Name of input file with extension
infile_names = ['mt washington.csv']
#Data is in US Customary Units

#Create start and end dates
start_date = '1948-01-01'
end_date = '2022-12-31'

#%% Load Data

#Set the working directory
os.chdir('C:\\Users\\nquer\\OneDrive\\Desktop\\ESCI Final Project')

#Load data and set index
data = pd.read_csv(infile_names[0], comment='#',
                   parse_dates=['DATE'], index_col=['DATE'])

#Drop unnecessary columns
columns=['SNOW','TMAX','TMIN']
data = data[columns]

#Cutting data to start and end dates
data=data[start_date:end_date]

#Drop NaN values from the dataset before analysis
data = data.dropna()

#%% Data Analysis

#Creating an average temperature column using min and max temperatures from each day
data['TAVG']=(data.TMIN+data.TMAX)/2

#Creating a new data set for just the winter season
#Create new columns in data for winter season and convert all nonwinter days to Nan

#%% Plot snow fall data
# Create plot to display snowfall data
fig, ax = plt.subplots()

# Plot snowfall data
ax.plot(data.SNOW, label='Snow Fall')  # x = dataframe index (date), y = snowfall in inches
                                     
# Add y-axis label    
ax.set_ylabel('Snow Fall (in)')   

# Add plot title
ax.set_title(fig_title1)

# Make x-tick labels diagonal to avoid overlap
fig.autofmt_xdate()  

#Add a legend to the plot
plt.legend()

#%% Plot temperature data
# Create plot to display temperature data
fig, ax2 = plt.subplots()

# Plot average daily temperature data
ax2.plot(data.TAVG, label='Average Daily Temperature') 

# Add y-axis label    
ax2.set_ylabel('Temperature (F)')   

# Add plot title
ax2.set_title(fig_title2)

# Optional command to make x-tick labels diagonal to avoid overlap
fig.autofmt_xdate()

#Add a legend to the plot
ax2.legend()

# Show the plot
plt.show()

Future Code:
#Cut data to just winter months
#Plot winter month data for both temperature and snowfall
#Average temperature and calculate total snowfall for each winter
#Plot winter average temp and total snowfall for each year with line of best fit for both (1 data point per year to make the graphs easier to read)
#Calculate statistics on line of best fits to determine p-value


Conclusion
    Discuss line of best fit slope (should be positive for temp and negative for snowfall if hypothesis is correct)
    Summarize any significance in average temperature change (p-value)
    Summarize any significance in total snowfall change (p-value)
    Discuss unexpected findings
    Possible error/ways to reduce error
    Implications of my findings
    Was my hypothesis correct?
    Conclusions that can be drawn
    